# ESPEI

### Extensible Self-optimizating Phase Equilibria Infrastructure

Documentation for internal and external APIs can be found at https://espei.org

Solutions to this notebook can be found at https://github.com/materialsgenomefoundation/2020-workshop-material

## Markov Chain Monte Carlo (MCMC)

### Running MCMC

The most minimal MCMC settings file that could be used for ESPEI only requires setting the source of the database (i.e. the database from parameter selection) and the number of iterations.

```yaml
system:
  phase_models: phases.json
  datasets: input-data/run
output:
  output_db:  mcmc.tdb
  verbosity:  2
mcmc:
  iterations: 100
  input_db: dft-aicc_penalty.tdb
```

By default, ESPEI will run in parallel using the `dask` package. If you try to run this locally, you may need to do an [extra step to configure dask](https://espei.org/en/latest/installation.html#configuration).

However, since we are using limited and shared cloud resources, we will make some compromises in terms of accuracy and storage. The settings we'll use for running MCMC simulations are as follows (saved as `mcmc_settings.yaml`):

All MCMC options are explained in [ESPEI's YAML settings file documentation](https://espei.org/en/latest/writing_input.html#mcmc). Compared to parameter generation, the options are more extensive and worth being familiar with.

In [1]:
import dask
import distributed
dask.config.set({'distributed.scheduler.work-stealing': False})

import yaml
from espei import run_espei
from pycalphad import Database, binplot, equilibrium, variables as v

In [2]:
with open('mcmc_settings.yaml') as fp:
    mcmc_settings = yaml.safe_load(fp)

Now we'll run the MCMC simulation for just two iterations. The outputs are the database which has the most optimal parameters of all samples and an `emcee.EnsembleSampler` object that contains the trace (contains samples of the parameters for every chain and iteration) and the log-likelihood.

In [3]:
dbf_mcmc, sampler = run_espei(mcmc_settings)

INFO:root:espei version       0.7.8
INFO:root:If you use ESPEI for work presented in a publication, we ask that you cite the following paper:
    B. Bocklund, R. Otis, A. Egorov, A. Obaied, I. Roslyakova, Z.-K. Liu, ESPEI for efficient thermodynamic database development, modification, and uncertainty quantification: application to Cu-Mg, MRS Commun. (2019) 1-10. doi:10.1557/mrc.2019.59.
TRACE:root:Loading and checking datasets.
TRACE:root:Finished checking datasets


Single phase datasets are present, but there are no specified `excluded_model_contributions` keys present. 'idmix' exclusion will be added automatically for backwards compatibility, but this will go away in ESPEI v0.8. If you want ideal mixing contributions to be excluded, see the documentation for building datasets: http://espei.org/en/latest/input_data.html


ValueError: The parameter 'work-stealing' is on in dask. Enabling this parameter causes some instability. Set 'distributed.scheduler.work-stealing: False' in your dask configuration. Configuration files on this machine are:
[] (latter files have priority).
See the example at http://espei.org/en/latest/installation.html#configuration for more.

Again, the key output from the sampler are the trace (`emcee` calls this the "chain") and the log-probability (lnprob). The trace has the shape `(number of chains, number of iterations, number of parameters)`. The log-probability has the shape  `(number of chains, number of iterations)`.



In [ ]:
trace = sampler.chain
lnprob = sampler.lnprobability

print(f"Trace shape:           {trace.shape}")
print(f"Log-probability shape: {lnprob.shape}")

With the MCMC simulation complete, we can see what the phase diagram looks like.

In [ ]:
from espei.plot import dataplot
from espei.datasets import recursive_glob, load_datasets

# load our JSON datasets into an in-memory database
datasets = load_datasets(recursive_glob('input-data', '*.json'))

In [ ]:
comps = ['CU', 'MG', 'VA']
phases = ['LIQUID', 'FCC_A1', 'HCP_A3', 'CUMG2', 'LAVES_C15']
conds = {v.P: 101325, v.T: (300, 1500, 10), v.X('MG'): (0, 1, 0.01)}
ax = binplot(dbf_mcmc, comps, phases, conds)
dataplot(comps, phases, conds, datasets, ax=ax)

After just three MCMC iterations through 24 chains, the phase diagram shows a significant improvement.

### Simulated MCMC analysis

Typically, a fully converged MCMC simulation with enough samples to do uncertainty quantification will require a few hundred to a few thousand calculations. 


Since performing an MCMC simulation for a significant period of time is not possible in this workshop, an existing pre-computed trace and log-probability are loaded that took 2000 iterations of sampling 24 chains for a total of 48,000 samples in parameter space. These 48,000 samples took 3.5 hours to run across 6 cores on a 2015 MacBook Pro (2.2 GHz Intel i7).


In [ ]:
import numpy as np
from espei.analysis import truncate_arrays

In [ ]:
trace = np.load('trace.npy')
lnprob = np.load('lnprob.npy')

trace, lnprob = truncate_arrays(trace, lnprob)

#### Visualizing convergence of the simulation

In [ ]:
import matplotlib.pyplot as plt
plt.plot(lnprob.T)
plt.title('Log-probability convergence\n(1 line = 1 chain)')
plt.xlabel('Iterations')
plt.ylabel('Log-probability')

plt.figure()
plt.plot(lnprob.T)
plt.title('Zoomed Log-probability convergence\n(1 line = 1 chain)')
plt.xlabel('Iterations')
plt.ylabel('Log-probability')
plt.ylim(-4000, -2000)

#### Visualizing change in a particular parameter

In [ ]:
# index of parameter of interest within the chain
# could be looped to produce figures for all parameters
parameter_idx = 5

num_chains = trace.shape[0]
ax = plt.figure().gca()
ax.set_xlabel('Iterations')
ax.set_ylabel('Parameter value')
ax.plot(trace[..., parameter_idx].T)
ax.set_title('Parameter Convergence')

#### Finding the optimal set of parameters and plotting the phase diagram

An MCMC simulation has many samples, but we are still likely interested in getting the set of parameters that's the best point estimate of the data.

ESPEI provides an `optimal_parameters` function that will extract the parameter set with the highest log-probability, which can be used to update the symbols that we fit in the database ($ \mathrm{VV0001} $, ...)..

In [ ]:
from espei.utils import database_symbols_to_fit, optimal_parameters
import copy

# make an in-memory copy of the database because updating the
# symbols with the optimal solutions will erase the old ones
dbf_opt = copy.deepcopy(dbf_mcmc)

# Find the optimal parameters and replace the values in the symbols dictionary
opt_params = dict(zip(database_symbols_to_fit(dbf_opt), optimal_parameters(trace, lnprob)))
dbf_opt.symbols.update(opt_params)

In [ ]:
# plot the phase diagram

ax = binplot(dbf_opt, comps, phases, conds)
dataplot(comps, phases, conds, datasets, ax=ax)

## Challenge yourself

Test your understanding by 

* Changing the relative weights of the types of data (`ZPF`, `SM` or try adding `HM`) to see what happens to the phase diagram after a few iterations.
* Change the verbosity in the MCMC settings YAML files to a higher or lower number and investigate the output. **Warning: a lot of output can be generated for verbosity level of 3, so it is best to try only 1-2 iterations**.
* Pick other parameter indices and try to match up the convergence of the log-probability with the changes in local minima for a parameter plotted across a number of iterations